In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from xgboost import plot_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MultiLabelBinarizer
from catboost import CatBoostClassifier





# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/music-vibes-datathon-fall23/sample_submission.csv
/kaggle/input/music-vibes-datathon-fall23/meta_data_2.csv
/kaggle/input/music-vibes-datathon-fall23/train.csv
/kaggle/input/music-vibes-datathon-fall23/test.csv
/kaggle/input/music-vibes-datathon-fall23/meta_data_1.csv


In [2]:
train_df = pd.read_csv("/kaggle/input/music-vibes-datathon-fall23/train.csv")
train_df.shape

(4961, 6)

In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4961 entries, 0 to 4960
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   song_id  4961 non-null   object
 1   album    4961 non-null   object
 2   artist   4961 non-null   object
 3   lyrics   4303 non-null   object
 4   track    4961 non-null   object
 5   target   4961 non-null   object
dtypes: object(6)
memory usage: 232.7+ KB


In [4]:
meta_data_1_df= pd.read_csv("/kaggle/input/music-vibes-datathon-fall23/meta_data_1.csv")
meta_data_1_df.shape

(8269, 18)

In [5]:
meta_data_2_df= pd.read_csv("/kaggle/input/music-vibes-datathon-fall23/meta_data_2.csv")
meta_data_2_df.shape

(8269, 51)

In [6]:
train_df.head(5)

,song_id,album,artist,lyrics,track,target
0,AA5aMeYP1klLv1BA,Stormy Monday Blues (Sun Records 70th / Remast...,T-Bone Walker,NaN,That Evening Train - Remastered 2022,Blues
1,Wr3MLRGLm08yjrGN,For All The Dogs,Drake,59 ContributorsTranslationsEspañolEnglishPortu...,Bahamas Promises,R&B
2,XtEbP8zIOloM6r5I,Break Out (1983 Version - Expanded Edition),The Pointer Sisters,5 ContributorsInvestigation of the Ferguson Po...,"Automatic - 12"" Special Remix",Disco
3,NW5kIVzyoiV0zJmJ,Legends Never Die,Bad Wolves,3 ContributorsLegends Never Die Lyrics(Yea)\n\...,Legends Never Die,Rock
4,1uhYBNywheqCdzrg,Everything’s Not Lost,Gregory Porter,NaN,Everything’s Not Lost,Jazz


In [7]:
left_join_train_meta_data_1 = train_df.merge(meta_data_1_df, left_on='song_id',right_on='id',how='left')
left_join_train_meta_data_1.head(5)

,song_id,album,artist,lyrics,track,target,id,adaptibility,danceability,duration,...,loudness,mode,naturality,positiveness,release_date,reputation,speechiness,tempo,time_signature,total_tracks
0,AA5aMeYP1klLv1BA,Stormy Monday Blues (Sun Records 70th / Remast...,T-Bone Walker,NaN,That Evening Train - Remastered 2022,Blues,AA5aMeYP1klLv1BA,5,58.8,372005.0,...,-15.290,5,0.85800,0.398,1970,32,NaN,95.061,3,12
1,Wr3MLRGLm08yjrGN,For All The Dogs,Drake,59 ContributorsTranslationsEspañolEnglishPortu...,Bahamas Promises,R&B,Wr3MLRGLm08yjrGN,1,62.6,368183.0,...,-15.794,6,0.85000,0.369,2023-10-06,83,0.0433,117.940,4,23
2,XtEbP8zIOloM6r5I,Break Out (1983 Version - Expanded Edition),The Pointer Sisters,5 ContributorsInvestigation of the Ferguson Po...,"Automatic - 12"" Special Remix",Disco,XtEbP8zIOloM6r5I,5,77.6,732725.0,...,-12.622,6,0.02360,0.915,1983-04-01,40,0.0662,111.116,4,16
3,NW5kIVzyoiV0zJmJ,Legends Never Die,Bad Wolves,3 ContributorsLegends Never Die Lyrics(Yea)\n\...,Legends Never Die,Rock,NW5kIVzyoiV0zJmJ,5,37.1,355317.0,...,-4.108,6,0.00139,0.319,2023-08-18,55,0.1500,160.023,4,2
4,1uhYBNywheqCdzrg,Everything’s Not Lost,Gregory Porter,NaN,Everything’s Not Lost,Jazz,1uhYBNywheqCdzrg,7,44.2,497383.0,...,-15.530,6,0.56700,0.346,2023-09-22,50,0.0347,170.725,3,1


In [8]:
final_df = left_join_train_meta_data_1.merge(meta_data_2_df,on='id',how='left')
final_df.head(5)

,song_id,album,artist,lyrics,track,target,id,adaptibility,danceability,duration,...,a46,a47,a48,a49,a5,a50,a6,a7,a8,a9
0,AA5aMeYP1klLv1BA,Stormy Monday Blues (Sun Records 70th / Remast...,T-Bone Walker,NaN,That Evening Train - Remastered 2022,Blues,AA5aMeYP1klLv1BA,5,58.8,372005.0,...,1.283386e-04,-741.5650,176.2173,0.796,-233.608,NaN,97.472,38.451403,NaN,2.57400
1,Wr3MLRGLm08yjrGN,For All The Dogs,Drake,59 ContributorsTranslationsEspañolEnglishPortu...,Bahamas Promises,R&B,Wr3MLRGLm08yjrGN,1,62.6,368183.0,...,1.110734e-07,-488.0346,187.4490,0.738,-248.924,0.2948,14.376,3.251403,0.02165,2.55000
2,XtEbP8zIOloM6r5I,Break Out (1983 Version - Expanded Edition),The Pointer Sisters,5 ContributorsInvestigation of the Ferguson Po...,"Automatic - 12"" Special Remix",Disco,XtEbP8zIOloM6r5I,5,77.6,732725.0,...,1.646928e-07,-844.4118,232.5411,1.830,-306.740,0.0804,88.576,75.251403,0.03310,0.07080
3,NW5kIVzyoiV0zJmJ,Legends Never Die,Bad Wolves,3 ContributorsLegends Never Die Lyrics(Yea)\n\...,Legends Never Die,Rock,NW5kIVzyoiV0zJmJ,5,37.1,355317.0,...,1.537279e-07,-376.2928,110.8980,0.638,-147.124,-0.0640,64.811,124.651403,0.07500,0.00417
4,1uhYBNywheqCdzrg,Everything’s Not Lost,Gregory Porter,NaN,Everything’s Not Lost,Jazz,1uhYBNywheqCdzrg,7,44.2,497383.0,...,4.065017e-06,-518.7020,131.8410,0.692,-175.416,0.8732,50.888,8.251403,0.01735,1.70100


In [9]:
column_missing_values = final_df.isnull().sum()
print(column_missing_values)

song_id      0
album        0
artist       0
lyrics     658
track        0
          ... 
a50        141
a6         188
a7         298
a8         141
a9           0
Length: 74, dtype: int64


In [10]:
row_missing_values = final_df.isnull().sum(axis=1).sort_values(ascending=False)
print(row_missing_values)

818     32
4024    31
1548    30
2071    27
4805    27
        ..
2793     0
1067     0
2795     0
2796     0
4960     0
Length: 4961, dtype: int64


In [11]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4961 entries, 0 to 4960
Data columns (total 74 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_id           4961 non-null   object 
 1   album             4961 non-null   object 
 2   artist            4961 non-null   object 
 3   lyrics            4303 non-null   object 
 4   track             4961 non-null   object 
 5   target            4961 non-null   object 
 6   id                4961 non-null   object 
 7   adaptibility      4961 non-null   int64  
 8   danceability      4773 non-null   float64
 9   duration          4886 non-null   float64
 10  energy            4663 non-null   float64
 11  explicit          4961 non-null   bool   
 12  happening         4961 non-null   float64
 13  instrumentalness  4961 non-null   float64
 14  loudness          4547 non-null   float64
 15  mode              4961 non-null   int64  
 16  naturality        4961 non-null   float64


In [12]:
final_df.shape

(4961, 74)

In [13]:
final_df.describe()

,adaptibility,danceability,duration,energy,happening,instrumentalness,loudness,mode,naturality,positiveness,...,a46,a47,a48,a49,a5,a50,a6,a7,a8,a9
count,4961.000000,4773.00000,4.886000e+03,4663.000000,4961.000000,4961.000000,4547.000000,4961.000000,4961.000000,4801.000000,...,4.754000e+03,4270.000000,4773.000000,4801.000000,4616.000000,4820.000000,4773.000000,4663.000000,4820.000000,4961.000000
mean,5.288248,56.28164,4.573025e+05,59.024925,0.176483,0.202083,-13.960227,5.590607,0.315425,0.457887,...,inf,-530.218227,168.314409,0.915774,-223.471390,0.382340,49.003892,59.501254,0.040479,0.946274
std,3.558435,18.63631,1.802009e+05,27.435181,0.138489,0.340984,12.920118,0.491771,0.344506,0.260304,...,NaN,286.219905,55.937303,0.520608,74.101293,0.611704,29.324582,54.870362,0.038636,1.033517
min,0.000000,0.00000,1.060710e+05,0.002030,0.015900,0.000000,-83.812000,5.000000,0.000001,0.000000,...,0.000000e+00,-2175.064800,-0.786000,0.000000,-385.916000,-2.632000,0.000000,-58.544537,0.000000,0.000004
25%,2.000000,44.50000,3.526890e+05,40.800000,0.094600,0.000003,-17.292000,5.000000,0.015200,0.238000,...,2.082383e-08,-686.789100,132.901800,0.476000,-278.371000,0.102300,22.789000,23.051403,0.018750,0.045600
50%,5.000000,58.20000,4.283780e+05,62.900000,0.119000,0.000924,-9.794000,6.000000,0.150000,0.446000,...,7.656592e-06,-507.722000,173.796000,0.892000,-230.726000,0.269200,48.408000,67.251403,0.025400,0.450000
75%,8.000000,70.10000,5.160395e+05,81.900000,0.219000,0.255000,-5.645000,6.000000,0.606000,0.662000,...,2.134259e-03,-345.869800,209.685000,1.324000,-177.024000,0.560100,75.000000,105.251403,0.044050,1.818000
max,11.000000,97.10000,3.245155e+06,100.000000,0.982000,0.995000,3.710000,6.000000,0.996000,0.979000,...,inf,367.661000,290.916000,1.958000,0.000000,3.665200,99.968000,141.451403,0.399000,2.988000


In [14]:
final_df.isna()

,song_id,album,artist,lyrics,track,target,id,adaptibility,danceability,duration,...,a46,a47,a48,a49,a5,a50,a6,a7,a8,a9
0,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4956,False,False,False,False,False,False,False,False,True,False,...,False,False,True,False,True,False,True,False,False,False
4957,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4958,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4959,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False


In [15]:
numerical_df = final_df.select_dtypes(include=['int64', 'float64'])
print(numerical_df.columns)
categorical_df = final_df.select_dtypes(exclude=['int64', 'float64'])
print(categorical_df.columns)
categorical_df.head(5)

Index(['adaptibility', 'danceability', 'duration', 'energy', 'happening',
       'instrumentalness', 'loudness', 'mode', 'naturality', 'positiveness',
       'reputation', 'speechiness', 'tempo', 'time_signature', 'total_tracks',
       'a1', 'a10', 'a11', 'a12', 'a13', 'a14', 'a15', 'a16', 'a17', 'a18',
       'a19', 'a2', 'a20', 'a21', 'a22', 'a23', 'a24', 'a25', 'a26', 'a27',
       'a28', 'a29', 'a3', 'a30', 'a31', 'a32', 'a33', 'a34', 'a35', 'a36',
       'a37', 'a38', 'a39', 'a4', 'a40', 'a41', 'a42', 'a43', 'a44', 'a45',
       'a46', 'a47', 'a48', 'a49', 'a5', 'a50', 'a6', 'a7', 'a8', 'a9'],
      dtype='object')
Index(['song_id', 'album', 'artist', 'lyrics', 'track', 'target', 'id',
       'explicit', 'release_date'],
      dtype='object')


,song_id,album,artist,lyrics,track,target,id,explicit,release_date
0,AA5aMeYP1klLv1BA,Stormy Monday Blues (Sun Records 70th / Remast...,T-Bone Walker,NaN,That Evening Train - Remastered 2022,Blues,AA5aMeYP1klLv1BA,False,1970
1,Wr3MLRGLm08yjrGN,For All The Dogs,Drake,59 ContributorsTranslationsEspañolEnglishPortu...,Bahamas Promises,R&B,Wr3MLRGLm08yjrGN,True,2023-10-06
2,XtEbP8zIOloM6r5I,Break Out (1983 Version - Expanded Edition),The Pointer Sisters,5 ContributorsInvestigation of the Ferguson Po...,"Automatic - 12"" Special Remix",Disco,XtEbP8zIOloM6r5I,False,1983-04-01
3,NW5kIVzyoiV0zJmJ,Legends Never Die,Bad Wolves,3 ContributorsLegends Never Die Lyrics(Yea)\n\...,Legends Never Die,Rock,NW5kIVzyoiV0zJmJ,True,2023-08-18
4,1uhYBNywheqCdzrg,Everything’s Not Lost,Gregory Porter,NaN,Everything’s Not Lost,Jazz,1uhYBNywheqCdzrg,False,2023-09-22


In [16]:
label_encoder = LabelEncoder()
for column in categorical_df.columns:
  encoded_column = '{}_encoded'.format(column)
  categorical_df[encoded_column] = label_encoder.fit_transform(categorical_df[column])

In [17]:
categorical_df.head(5)

,song_id,album,artist,lyrics,track,target,id,explicit,release_date,song_id_encoded,album_encoded,artist_encoded,lyrics_encoded,track_encoded,target_encoded,id_encoded,explicit_encoded,release_date_encoded
0,AA5aMeYP1klLv1BA,Stormy Monday Blues (Sun Records 70th / Remast...,T-Bone Walker,NaN,That Evening Train - Remastered 2022,Blues,AA5aMeYP1klLv1BA,False,1970,906,3349,2711,4050,3871,1,906,0,101
1,Wr3MLRGLm08yjrGN,For All The Dogs,Drake,59 ContributorsTranslationsEspañolEnglishPortu...,Bahamas Promises,R&B,Wr3MLRGLm08yjrGN,True,2023-10-06,2718,1246,818,3395,359,13,2718,1,1853
2,XtEbP8zIOloM6r5I,Break Out (1983 Version - Expanded Edition),The Pointer Sisters,5 ContributorsInvestigation of the Ferguson Po...,"Automatic - 12"" Special Remix",Disco,XtEbP8zIOloM6r5I,False,1983-04-01,2818,544,2869,3195,297,4,2818,0,272
3,NW5kIVzyoiV0zJmJ,Legends Never Die,Bad Wolves,3 ContributorsLegends Never Die Lyrics(Yea)\n\...,Legends Never Die,Rock,NW5kIVzyoiV0zJmJ,True,2023-08-18,1971,2021,253,2352,2221,14,1971,1,1808
4,1uhYBNywheqCdzrg,Everything’s Not Lost,Gregory Porter,NaN,Everything’s Not Lost,Jazz,1uhYBNywheqCdzrg,False,2023-09-22,304,1148,1132,4050,1208,9,304,0,1839


In [18]:
categorical_columns = ['song_id_encoded', 'album_encoded', 'artist_encoded', 'lyrics_encoded', 'track_encoded', 'explicit_encoded', 'release_date_encoded']
categorical_df_encoded = categorical_df[categorical_columns]

In [19]:
categorical_df_encoded.columns

Index(['song_id_encoded', 'album_encoded', 'artist_encoded', 'lyrics_encoded',
       'track_encoded', 'explicit_encoded', 'release_date_encoded'],
      dtype='object')

In [20]:
categorical_df_encoded.head(5)

,song_id_encoded,album_encoded,artist_encoded,lyrics_encoded,track_encoded,explicit_encoded,release_date_encoded
0,906,3349,2711,4050,3871,0,101
1,2718,1246,818,3395,359,1,1853
2,2818,544,2869,3195,297,0,272
3,1971,2021,253,2352,2221,1,1808
4,304,1148,1132,4050,1208,0,1839


In [21]:
numerical_columns_for_imputation = [col for col in numerical_df.columns]
print(numerical_columns_for_imputation)

['adaptibility', 'danceability', 'duration', 'energy', 'happening', 'instrumentalness', 'loudness', 'mode', 'naturality', 'positiveness', 'reputation', 'speechiness', 'tempo', 'time_signature', 'total_tracks', 'a1', 'a10', 'a11', 'a12', 'a13', 'a14', 'a15', 'a16', 'a17', 'a18', 'a19', 'a2', 'a20', 'a21', 'a22', 'a23', 'a24', 'a25', 'a26', 'a27', 'a28', 'a29', 'a3', 'a30', 'a31', 'a32', 'a33', 'a34', 'a35', 'a36', 'a37', 'a38', 'a39', 'a4', 'a40', 'a41', 'a42', 'a43', 'a44', 'a45', 'a46', 'a47', 'a48', 'a49', 'a5', 'a50', 'a6', 'a7', 'a8', 'a9']


In [22]:
knn_imputer = KNNImputer(n_neighbors=5)
numerical_df.replace([np.inf, -np.inf], np.nan, inplace=True)
numerical_imputed_array = knn_imputer.fit_transform(numerical_df)
numerical_imputed_df = pd.DataFrame(numerical_imputed_array, columns=numerical_df.columns)
modes = numerical_df.mode().iloc[0]
for col in numerical_imputed_df.columns:
    numerical_df[col].fillna(modes[col], inplace=True)

In [23]:
print(numerical_imputed_df)

      adaptibility  danceability  duration  energy  happening  \
0              5.0         58.80  372005.0    48.5     0.0609   
1              1.0         62.60  368183.0    30.9     0.1170   
2              5.0         77.60  732725.0    66.9     0.0863   
3              5.0         37.10  355317.0    91.6     0.1340   
4              7.0         44.20  497383.0    33.4     0.2530   
...            ...           ...       ...     ...        ...   
4956           7.0         58.36  404993.0    43.4     0.2630   
4957           6.0         83.00  469755.0    53.1     0.2930   
4958           6.0         68.70  279471.0    63.0     0.2210   
4959           4.0         36.50  497933.0    27.3     0.1090   
4960          11.0         14.80  210723.0    10.1     0.1370   

      instrumentalness  loudness  mode  naturality  positiveness  ...  \
0             0.012200   -15.290   5.0     0.85800         0.398  ...   
1             0.000013   -15.794   6.0     0.85000         0.369  ...   


In [24]:
numerical_imputed_df = pd.DataFrame(numerical_imputed_df, columns=numerical_columns_for_imputation)

In [25]:
numerical_imputed_df.head(5)

,adaptibility,danceability,duration,energy,happening,instrumentalness,loudness,mode,naturality,positiveness,...,a46,a47,a48,a49,a5,a50,a6,a7,a8,a9
0,5.0,58.8,372005.0,48.5,0.0609,0.012200,-15.290,5.0,0.85800,0.398,...,1.283386e-04,-741.5650,176.2173,0.796,-233.608,0.42744,97.472,38.451403,0.03468,2.57400
1,1.0,62.6,368183.0,30.9,0.1170,0.000013,-15.794,6.0,0.85000,0.369,...,1.110734e-07,-488.0346,187.4490,0.738,-248.924,0.29480,14.376,3.251403,0.02165,2.55000
2,5.0,77.6,732725.0,66.9,0.0863,0.000018,-12.622,6.0,0.02360,0.915,...,1.646928e-07,-844.4118,232.5411,1.830,-306.740,0.08040,88.576,75.251403,0.03310,0.07080
3,5.0,37.1,355317.0,91.6,0.1340,0.000025,-4.108,6.0,0.00139,0.319,...,1.537279e-07,-376.2928,110.8980,0.638,-147.124,-0.06400,64.811,124.651403,0.07500,0.00417
4,7.0,44.2,497383.0,33.4,0.2530,0.000694,-15.530,6.0,0.56700,0.346,...,4.065017e-06,-518.7020,131.8410,0.692,-175.416,0.87320,50.888,8.251403,0.01735,1.70100


In [26]:
numerical_imputed_df.shape

(4961, 65)

In [27]:
numerical_imputed_df['a46'].isna()

0       False
1       False
2       False
3       False
4       False
        ...  
4956    False
4957    False
4958    False
4959    False
4960    False
Name: a46, Length: 4961, dtype: bool

In [28]:
final_df.head(5)

,song_id,album,artist,lyrics,track,target,id,adaptibility,danceability,duration,...,a46,a47,a48,a49,a5,a50,a6,a7,a8,a9
0,AA5aMeYP1klLv1BA,Stormy Monday Blues (Sun Records 70th / Remast...,T-Bone Walker,NaN,That Evening Train - Remastered 2022,Blues,AA5aMeYP1klLv1BA,5,58.8,372005.0,...,1.283386e-04,-741.5650,176.2173,0.796,-233.608,NaN,97.472,38.451403,NaN,2.57400
1,Wr3MLRGLm08yjrGN,For All The Dogs,Drake,59 ContributorsTranslationsEspañolEnglishPortu...,Bahamas Promises,R&B,Wr3MLRGLm08yjrGN,1,62.6,368183.0,...,1.110734e-07,-488.0346,187.4490,0.738,-248.924,0.2948,14.376,3.251403,0.02165,2.55000
2,XtEbP8zIOloM6r5I,Break Out (1983 Version - Expanded Edition),The Pointer Sisters,5 ContributorsInvestigation of the Ferguson Po...,"Automatic - 12"" Special Remix",Disco,XtEbP8zIOloM6r5I,5,77.6,732725.0,...,1.646928e-07,-844.4118,232.5411,1.830,-306.740,0.0804,88.576,75.251403,0.03310,0.07080
3,NW5kIVzyoiV0zJmJ,Legends Never Die,Bad Wolves,3 ContributorsLegends Never Die Lyrics(Yea)\n\...,Legends Never Die,Rock,NW5kIVzyoiV0zJmJ,5,37.1,355317.0,...,1.537279e-07,-376.2928,110.8980,0.638,-147.124,-0.0640,64.811,124.651403,0.07500,0.00417
4,1uhYBNywheqCdzrg,Everything’s Not Lost,Gregory Porter,NaN,Everything’s Not Lost,Jazz,1uhYBNywheqCdzrg,7,44.2,497383.0,...,4.065017e-06,-518.7020,131.8410,0.692,-175.416,0.8732,50.888,8.251403,0.01735,1.70100


In [29]:
final_df.shape

(4961, 74)

In [30]:
numerical_imputed_df.head(5)

,adaptibility,danceability,duration,energy,happening,instrumentalness,loudness,mode,naturality,positiveness,...,a46,a47,a48,a49,a5,a50,a6,a7,a8,a9
0,5.0,58.8,372005.0,48.5,0.0609,0.012200,-15.290,5.0,0.85800,0.398,...,1.283386e-04,-741.5650,176.2173,0.796,-233.608,0.42744,97.472,38.451403,0.03468,2.57400
1,1.0,62.6,368183.0,30.9,0.1170,0.000013,-15.794,6.0,0.85000,0.369,...,1.110734e-07,-488.0346,187.4490,0.738,-248.924,0.29480,14.376,3.251403,0.02165,2.55000
2,5.0,77.6,732725.0,66.9,0.0863,0.000018,-12.622,6.0,0.02360,0.915,...,1.646928e-07,-844.4118,232.5411,1.830,-306.740,0.08040,88.576,75.251403,0.03310,0.07080
3,5.0,37.1,355317.0,91.6,0.1340,0.000025,-4.108,6.0,0.00139,0.319,...,1.537279e-07,-376.2928,110.8980,0.638,-147.124,-0.06400,64.811,124.651403,0.07500,0.00417
4,7.0,44.2,497383.0,33.4,0.2530,0.000694,-15.530,6.0,0.56700,0.346,...,4.065017e-06,-518.7020,131.8410,0.692,-175.416,0.87320,50.888,8.251403,0.01735,1.70100


In [31]:
numerical_imputed_df['key'] = range(1, len(numerical_imputed_df) + 1)

In [32]:
numerical_imputed_df.head(5)

,adaptibility,danceability,duration,energy,happening,instrumentalness,loudness,mode,naturality,positiveness,...,a47,a48,a49,a5,a50,a6,a7,a8,a9,key
0,5.0,58.8,372005.0,48.5,0.0609,0.012200,-15.290,5.0,0.85800,0.398,...,-741.5650,176.2173,0.796,-233.608,0.42744,97.472,38.451403,0.03468,2.57400,1
1,1.0,62.6,368183.0,30.9,0.1170,0.000013,-15.794,6.0,0.85000,0.369,...,-488.0346,187.4490,0.738,-248.924,0.29480,14.376,3.251403,0.02165,2.55000,2
2,5.0,77.6,732725.0,66.9,0.0863,0.000018,-12.622,6.0,0.02360,0.915,...,-844.4118,232.5411,1.830,-306.740,0.08040,88.576,75.251403,0.03310,0.07080,3
3,5.0,37.1,355317.0,91.6,0.1340,0.000025,-4.108,6.0,0.00139,0.319,...,-376.2928,110.8980,0.638,-147.124,-0.06400,64.811,124.651403,0.07500,0.00417,4
4,7.0,44.2,497383.0,33.4,0.2530,0.000694,-15.530,6.0,0.56700,0.346,...,-518.7020,131.8410,0.692,-175.416,0.87320,50.888,8.251403,0.01735,1.70100,5


In [33]:
categorical_df_encoded.head(5)

,song_id_encoded,album_encoded,artist_encoded,lyrics_encoded,track_encoded,explicit_encoded,release_date_encoded
0,906,3349,2711,4050,3871,0,101
1,2718,1246,818,3395,359,1,1853
2,2818,544,2869,3195,297,0,272
3,1971,2021,253,2352,2221,1,1808
4,304,1148,1132,4050,1208,0,1839


In [34]:
categorical_df_encoded['key'] = range(1, len(categorical_df_encoded) + 1)

/tmp/ipykernel_20/576405888.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categorical_df_encoded['key'] = range(1, len(categorical_df_encoded) + 1)


In [35]:
categorical_df_encoded.head(5)

,song_id_encoded,album_encoded,artist_encoded,lyrics_encoded,track_encoded,explicit_encoded,release_date_encoded,key
0,906,3349,2711,4050,3871,0,101,1
1,2718,1246,818,3395,359,1,1853,2
2,2818,544,2869,3195,297,0,272,3
3,1971,2021,253,2352,2221,1,1808,4
4,304,1148,1132,4050,1208,0,1839,5


In [36]:
final_imputed_df = pd.merge(numerical_imputed_df, categorical_df_encoded, on='key', how='inner')

In [37]:
final_imputed_df.head(5)

,adaptibility,danceability,duration,energy,happening,instrumentalness,loudness,mode,naturality,positiveness,...,a8,a9,key,song_id_encoded,album_encoded,artist_encoded,lyrics_encoded,track_encoded,explicit_encoded,release_date_encoded
0,5.0,58.8,372005.0,48.5,0.0609,0.012200,-15.290,5.0,0.85800,0.398,...,0.03468,2.57400,1,906,3349,2711,4050,3871,0,101
1,1.0,62.6,368183.0,30.9,0.1170,0.000013,-15.794,6.0,0.85000,0.369,...,0.02165,2.55000,2,2718,1246,818,3395,359,1,1853
2,5.0,77.6,732725.0,66.9,0.0863,0.000018,-12.622,6.0,0.02360,0.915,...,0.03310,0.07080,3,2818,544,2869,3195,297,0,272
3,5.0,37.1,355317.0,91.6,0.1340,0.000025,-4.108,6.0,0.00139,0.319,...,0.07500,0.00417,4,1971,2021,253,2352,2221,1,1808
4,7.0,44.2,497383.0,33.4,0.2530,0.000694,-15.530,6.0,0.56700,0.346,...,0.01735,1.70100,5,304,1148,1132,4050,1208,0,1839


In [38]:
column_missing_values = final_imputed_df.isnull().sum()
print(column_missing_values)

adaptibility            0
danceability            0
duration                0
energy                  0
happening               0
                       ..
artist_encoded          0
lyrics_encoded          0
track_encoded           0
explicit_encoded        0
release_date_encoded    0
Length: 73, dtype: int64


In [39]:
row_missing_values = final_imputed_df.isnull().sum(axis=1).sort_values(ascending=False)
print(row_missing_values)

0       0
3305    0
3312    0
3311    0
3310    0
       ..
1653    0
1652    0
1651    0
1650    0
4960    0
Length: 4961, dtype: int64


In [40]:
y = categorical_df['target_encoded']
y.head(5)

0     1
1    13
2     4
3    14
4     9
Name: target_encoded, dtype: int64

In [41]:
X = final_imputed_df
X.shape

(4961, 73)

In [42]:
X.isna()

,adaptibility,danceability,duration,energy,happening,instrumentalness,loudness,mode,naturality,positiveness,...,a8,a9,key,song_id_encoded,album_encoded,artist_encoded,lyrics_encoded,track_encoded,explicit_encoded,release_date_encoded
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4956,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4957,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4958,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4959,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [43]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Initialize and train the CatBoost classifier
model = CatBoostClassifier(iterations=1000, depth=6, learning_rate=0.1, loss_function='MultiClass', verbose=100)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

test_accuracy = accuracy_score(y_test, y_pred)
print(test_accuracy)

0:	learn: 2.6129878	total: 189ms	remaining: 3m 8s
100:	learn: 0.8432874	total: 12.6s	remaining: 1m 51s
200:	learn: 0.6047439	total: 25.4s	remaining: 1m 40s
300:	learn: 0.4661126	total: 37.8s	remaining: 1m 27s
400:	learn: 0.3698672	total: 50.3s	remaining: 1m 15s
500:	learn: 0.2973781	total: 1m 3s	remaining: 1m 2s
600:	learn: 0.2429426	total: 1m 15s	remaining: 50.2s
700:	learn: 0.2006611	total: 1m 28s	remaining: 37.8s
800:	learn: 0.1683132	total: 1m 41s	remaining: 25.1s
900:	learn: 0.1426525	total: 1m 53s	remaining: 12.5s
999:	learn: 0.1223551	total: 2m 6s	remaining: 0us
0.6445115810674723


In [44]:
X_train, X_cv_test, y_train, y_cv_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_cv, X_test, y_cv, y_test = train_test_split(X_cv_test, y_cv_test, test_size=0.5, random_state=42)
print(y_train)

4685     6
3176    14
1650    14
3734     7
112      1
        ..
4426    14
466     15
3092     4
3772     7
860     15
Name: target_encoded, Length: 3472, dtype: int64


In [45]:
xg_clf = xgb.XGBClassifier(objective='multi:softprob', eval_metric='mlogloss',use_label_encoder=False)

/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


In [46]:
X.isna()

,adaptibility,danceability,duration,energy,happening,instrumentalness,loudness,mode,naturality,positiveness,...,a8,a9,key,song_id_encoded,album_encoded,artist_encoded,lyrics_encoded,track_encoded,explicit_encoded,release_date_encoded
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4956,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4957,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4958,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4959,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [47]:
xg_clf.fit(X_train, y_train)
y_cv_pred = xg_clf.predict(X_cv)

In [48]:
cv_accuracy = accuracy_score(y_cv, y_cv_pred)
print(f"Cross-Validation Accuracy: {cv_accuracy*100:.2f}%")

Cross-Validation Accuracy: 62.63%


In [49]:
y_test_pred = xg_clf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy*100:.2f}%")

Test Accuracy: 62.28%


In [50]:
f1 = f1_score(y_test, y_test_pred,average='weighted')
print(f1)

0.6192431771925898


In [51]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
# Train the classifier on the training data
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [52]:
y_pred = clf.predict(X_test)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.574496644295302


In [53]:
clf = KNeighborsClassifier(n_neighbors=25)

# Train the classifier on the training data
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.26711409395973157


In [54]:
test_df = pd.read_csv("/kaggle/input/music-vibes-datathon-fall23/test.csv")

In [55]:
left_join_train_meta_data_1 = test_df.merge(meta_data_1_df, left_on='song_id',right_on='id',how='left')
left_join_train_meta_data_1.head(5)

,song_id,album,artist,lyrics,track,id,adaptibility,danceability,duration,energy,...,loudness,mode,naturality,positiveness,release_date,reputation,speechiness,tempo,time_signature,total_tracks
0,7c61FpilqRU/3Ley,From A Room: Volume 1,Chris Stapleton,17 ContributorsBroken Halos Lyrics[Refrain]\nS...,Broken Halos,7c61FpilqRU/3Ley,8,61.3,361817.0,56.40,...,-9.170,6,0.134,0.538,2017-05-05,80,0.0251,78.388,4,9
1,EmqUjbC3coby/LZy,Vogue,Noizu,2 ContributorsVogue Lyrics[Verse]\nIt's giving...,Vogue,EmqUjbC3coby/LZy,6,74.5,384379.0,93.20,...,-4.984,6,0.111,0.928,2023-09-08,60,0.0507,128.000,4,2
2,lvF5H8aYwo+TlFJe,Silent Sky,Lofi Leif,3 ContributorsAll of the good songs except the...,Silent Sky,lvF5H8aYwo+TlFJe,2,69.7,279451.0,4.45,...,-40.320,6,0.925,0.163,2022-03-15,51,0.0399,116.028,4,1
3,O+oGRFmYSUbebxCK,Hold Space For Me,Orion Sun,5 ContributorsNe Me Quitte Pas (Don’t Leave Me...,Ne Me Quitte Pas (Don't Leave Me),O+oGRFmYSUbebxCK,6,61.7,337497.0,50.00,...,-6.494,6,0.386,0.516,2020-03-27,63,0.0340,90.026,4,11
4,rUR7HzUw1p41lUUn,ANTI (Deluxe),Rihanna,158 ContributorsTranslationsEspañolFrançaisNee...,Needed Me,rUR7HzUw1p41lUUn,5,67.1,383205.0,31.40,...,-11.182,5,0.110,0.296,2016-01-28,84,0.2440,110.898,4,16


In [56]:
final_test_df = left_join_train_meta_data_1.merge(meta_data_2_df,on='id',how='left')
final_test_df.head(5)

,song_id,album,artist,lyrics,track,id,adaptibility,danceability,duration,energy,...,a46,a47,a48,a49,a5,a50,a6,a7,a8,a9
0,7c61FpilqRU/3Ley,From A Room: Volume 1,Chris Stapleton,17 ContributorsBroken Halos Lyrics[Refrain]\nS...,Broken Halos,7c61FpilqRU/3Ley,8,61.3,361817.0,56.40,...,2.104914e-08,-517.1880,183.7245,1.076,-243.048,0.1336,46.397,54.251403,0.01255,0.402
1,EmqUjbC3coby/LZy,Vogue,Noizu,2 ContributorsVogue Lyrics[Verse]\nIt's giving...,Vogue,EmqUjbC3coby/LZy,6,74.5,384379.0,93.20,...,4.382812e-04,-464.5088,223.0920,1.856,-294.288,0.3412,93.625,127.851403,0.02535,0.333
2,lvF5H8aYwo+TlFJe,Silent Sky,Lofi Leif,3 ContributorsAll of the good songs except the...,Silent Sky,lvF5H8aYwo+TlFJe,2,69.7,279451.0,4.45,...,7.989451e-03,-179.4240,208.7910,0.326,-278.148,0.2524,8.873,-49.648597,0.01995,2.775
3,O+oGRFmYSUbebxCK,Hold Space For Me,Orion Sun,5 ContributorsNe Me Quitte Pas (Don’t Leave Me...,Ne Me Quitte Pas (Don't Leave Me),O+oGRFmYSUbebxCK,6,61.7,337497.0,50.00,...,5.787217e-05,-324.7000,184.7790,1.032,-244.736,0.2920,85.113,41.451403,0.01700,1.158
4,rUR7HzUw1p41lUUn,ANTI (Deluxe),Rihanna,158 ContributorsTranslationsEspañolFrançaisNee...,Needed Me,rUR7HzUw1p41lUUn,5,67.1,383205.0,31.40,...,0.000000e+00,-351.1148,201.0525,0.592,-267.216,-0.6460,11.711,4.251403,0.12200,0.330


In [57]:
numerical_test_df = final_test_df.select_dtypes(include=['int64', 'float64'])
categorical_test_df = final_test_df.select_dtypes(exclude=['int64', 'float64'])
print(numerical_test_df)
print(categorical_test_df)

      adaptibility  danceability  duration  energy  happening  \
0                8          61.3  361817.0   56.40     0.0585   
1                6          74.5  384379.0   93.20     0.1360   
2                2          69.7  279451.0    4.45     0.1030   
3                6          61.7  337497.0   50.00     0.1070   
4                5          67.1  383205.0   31.40     0.0825   
...            ...           ...       ...     ...        ...   
3303            11          93.6  227285.0   66.50     0.0972   
3304             0          58.7  379017.0   89.40     0.1790   
3305             9          55.4  380137.0   49.80     0.7950   
3306             4          72.7  478127.0   66.70     0.5750   
3307             0          59.6  563925.0   89.60     0.3720   

      instrumentalness  loudness  mode  naturality  positiveness  ...  \
0             0.000002    -9.170     6     0.13400        0.5380  ...   
1             0.056100    -4.984     6     0.11100        0.9280  ...   


In [58]:
label_encoder = LabelEncoder()

In [59]:
for column in categorical_test_df.columns:
  encoded_column = '{}_encoded'.format(column)
  categorical_test_df[encoded_column] = label_encoder.fit_transform(categorical_test_df[column])  

In [60]:
categorical_test_columns = ['song_id_encoded', 'album_encoded', 'artist_encoded', 'lyrics_encoded', 'track_encoded', 'explicit_encoded', 'release_date_encoded']
categorical_test_df_encoded = categorical_test_df[categorical_columns]

In [61]:
numerical_columns_for_test_imputation = [col for col in final_test_df.columns if col not in categorical_test_df]
print(numerical_columns_for_imputation)

['adaptibility', 'danceability', 'duration', 'energy', 'happening', 'instrumentalness', 'loudness', 'mode', 'naturality', 'positiveness', 'reputation', 'speechiness', 'tempo', 'time_signature', 'total_tracks', 'a1', 'a10', 'a11', 'a12', 'a13', 'a14', 'a15', 'a16', 'a17', 'a18', 'a19', 'a2', 'a20', 'a21', 'a22', 'a23', 'a24', 'a25', 'a26', 'a27', 'a28', 'a29', 'a3', 'a30', 'a31', 'a32', 'a33', 'a34', 'a35', 'a36', 'a37', 'a38', 'a39', 'a4', 'a40', 'a41', 'a42', 'a43', 'a44', 'a45', 'a46', 'a47', 'a48', 'a49', 'a5', 'a50', 'a6', 'a7', 'a8', 'a9']


In [62]:
knn_imputer = KNNImputer(n_neighbors=5)
numerical_test_df.replace([np.inf, -np.inf], np.nan, inplace=True)
numerical_imputed_test_array = knn_imputer.fit_transform(numerical_test_df)
numerical_imputed_test_df = pd.DataFrame(numerical_imputed_test_array, columns=numerical_test_df.columns)
modes = numerical_imputed_test_df.mode().iloc[0]
for col in numerical_imputed_test_df.columns:
    numerical_imputed_test_df[col].fillna(modes[col], inplace=True)

In [63]:
print(numerical_imputed_test_df)

      adaptibility  danceability  duration  energy  happening  \
0              8.0          61.3  361817.0   56.40     0.0585   
1              6.0          74.5  384379.0   93.20     0.1360   
2              2.0          69.7  279451.0    4.45     0.1030   
3              6.0          61.7  337497.0   50.00     0.1070   
4              5.0          67.1  383205.0   31.40     0.0825   
...            ...           ...       ...     ...        ...   
3303          11.0          93.6  227285.0   66.50     0.0972   
3304           0.0          58.7  379017.0   89.40     0.1790   
3305           9.0          55.4  380137.0   49.80     0.7950   
3306           4.0          72.7  478127.0   66.70     0.5750   
3307           0.0          59.6  563925.0   89.60     0.3720   

      instrumentalness  loudness  mode  naturality  positiveness  ...  \
0             0.000002   -9.1700   6.0     0.13400        0.5380  ...   
1             0.056100   -4.9840   6.0     0.11100        0.9280  ...   


In [64]:
numerical_imputed_test_df = pd.DataFrame(numerical_imputed_test_df, columns=numerical_columns_for_test_imputation)

In [65]:
numerical_imputed_test_df['key'] = range(1, len(numerical_imputed_test_df) + 1)

In [66]:
categorical_test_df_encoded['key'] = range(1, len(categorical_test_df) + 1)

/tmp/ipykernel_20/975999539.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categorical_test_df_encoded['key'] = range(1, len(categorical_test_df) + 1)


In [67]:
final_imputed_test_df = pd.merge(numerical_imputed_test_df, categorical_test_df_encoded, on='key', how='inner')

In [68]:
X_test_dataset = final_imputed_test_df

In [69]:


# Make predictions on the test set
y_test_dataset_pred = model.predict(X_test_dataset)

#y_test_dataset_pred = clf.predict(X_test_dataset)

In [70]:
ss = pd.read_csv("/kaggle/input/music-vibes-datathon-fall23/sample_submission.csv")

In [71]:
print(y_test_dataset_pred)
flat_list = np.array(y_test_dataset_pred).flatten()
print(flat_list)

[[ 3]
 [ 4]
 [10]
 ...
 [12]
 [ 5]
 [11]]
[ 3  4 10 ... 12  5 11]


In [72]:
label_encoding_dict = {
    0: 'Ambient',
    1: 'Blues',
    2: 'Classical',
	3: 'Country',
    4: 'Disco',
    5: 'EDM',
	6: 'Funk',
    7: 'HipHop',
    8: 'Indie',
	9: 'Jazz',
	10: 'Lofi',
    11: 'Metal',
    12: 'Pop',
	13: 'R&B',
    14: 'Rock',
    15: 'Soul',
}

In [73]:
target_encoded_labels = flat_list.tolist()
print(target_encoded_labels)

[3, 4, 10, 8, 7, 15, 3, 14, 14, 11, 11, 10, 3, 0, 3, 6, 6, 10, 1, 11, 13, 7, 6, 11, 9, 1, 0, 10, 6, 1, 14, 6, 13, 1, 5, 14, 0, 1, 10, 10, 5, 5, 9, 15, 12, 10, 8, 1, 2, 15, 2, 5, 7, 4, 3, 15, 12, 7, 11, 11, 3, 8, 12, 1, 1, 11, 10, 7, 13, 5, 3, 11, 1, 11, 14, 15, 6, 6, 14, 7, 6, 12, 6, 2, 6, 14, 11, 12, 11, 15, 9, 10, 1, 2, 3, 9, 1, 11, 1, 14, 2, 4, 8, 3, 11, 6, 3, 11, 13, 14, 2, 3, 11, 14, 14, 11, 14, 6, 5, 3, 5, 6, 4, 9, 1, 10, 5, 15, 5, 3, 3, 11, 0, 13, 1, 4, 15, 0, 2, 13, 1, 6, 2, 6, 14, 8, 5, 11, 11, 6, 9, 12, 14, 10, 14, 15, 0, 1, 11, 8, 1, 15, 6, 4, 11, 1, 13, 1, 12, 12, 13, 7, 11, 3, 4, 12, 5, 1, 5, 12, 11, 15, 15, 11, 0, 1, 3, 9, 6, 4, 8, 3, 11, 3, 5, 14, 4, 0, 11, 11, 3, 8, 5, 2, 11, 7, 11, 8, 4, 10, 10, 0, 1, 13, 12, 4, 4, 15, 8, 1, 15, 5, 3, 4, 8, 1, 13, 11, 4, 11, 5, 11, 14, 11, 4, 4, 15, 11, 5, 7, 0, 1, 0, 4, 3, 14, 11, 14, 1, 1, 3, 13, 14, 7, 1, 12, 3, 14, 11, 2, 11, 15, 14, 3, 3, 10, 4, 11, 14, 15, 13, 13, 12, 10, 1, 1, 9, 13, 13, 14, 4, 8, 15, 11, 10, 1, 10, 6, 1, 11, 15

In [74]:
original_labels = [label_encoding_dict[label] for label in target_encoded_labels ]
print(original_labels)

['Country', 'Disco', 'Lofi', 'Indie', 'HipHop', 'Soul', 'Country', 'Rock', 'Rock', 'Metal', 'Metal', 'Lofi', 'Country', 'Ambient', 'Country', 'Funk', 'Funk', 'Lofi', 'Blues', 'Metal', 'R&B', 'HipHop', 'Funk', 'Metal', 'Jazz', 'Blues', 'Ambient', 'Lofi', 'Funk', 'Blues', 'Rock', 'Funk', 'R&B', 'Blues', 'EDM', 'Rock', 'Ambient', 'Blues', 'Lofi', 'Lofi', 'EDM', 'EDM', 'Jazz', 'Soul', 'Pop', 'Lofi', 'Indie', 'Blues', 'Classical', 'Soul', 'Classical', 'EDM', 'HipHop', 'Disco', 'Country', 'Soul', 'Pop', 'HipHop', 'Metal', 'Metal', 'Country', 'Indie', 'Pop', 'Blues', 'Blues', 'Metal', 'Lofi', 'HipHop', 'R&B', 'EDM', 'Country', 'Metal', 'Blues', 'Metal', 'Rock', 'Soul', 'Funk', 'Funk', 'Rock', 'HipHop', 'Funk', 'Pop', 'Funk', 'Classical', 'Funk', 'Rock', 'Metal', 'Pop', 'Metal', 'Soul', 'Jazz', 'Lofi', 'Blues', 'Classical', 'Country', 'Jazz', 'Blues', 'Metal', 'Blues', 'Rock', 'Classical', 'Disco', 'Indie', 'Country', 'Metal', 'Funk', 'Country', 'Metal', 'R&B', 'Rock', 'Classical', 'Country', 

In [75]:
y_test_dataset_pred_original_labels = pd.DataFrame(original_labels, columns=['target'])
ss['target'] = y_test_dataset_pred_original_labels
ss

,song_id,target
0,7c61FpilqRU/3Ley,Country
1,EmqUjbC3coby/LZy,Disco
2,lvF5H8aYwo+TlFJe,Lofi
3,O+oGRFmYSUbebxCK,Indie
4,rUR7HzUw1p41lUUn,HipHop
...,...,...
3303,ObfXKLfo3N9IuZGw,Country
3304,qCxgC5trW/Xl/wC8,Indie
3305,z8dKvyoqkEVA1aKZ,Pop
3306,s2RNjtkc0Rzt5smL,EDM


In [76]:
ss.to_csv("submission.csv", index=False)